Now that we have corrected for motion and we have extracted the mean activity for each region over time, we can now do some interesting dynamic analysis. One of the aims of your thesis is to compare the results from the phase analysis to the analysis of sliding window approach. 

This papers ([paper 1](http://online.liebertpub.com/doi/abs/10.1089/brain.2011.0068) and [paper 2](http://www.sciencedirect.com/science/article/pii/S1053811916307881)) will give you a better idea of how both analysis method work and explain their differences.

We will start by looking at the phase synchrony.


# Phase Analysis

In [2]:
# Import the required libraries
from scipy.signal import hilbert
import numpy as np
import os

## Hilbert Transform

The first step of the Phase Analyis consits in doing the Hilbert Transform on the dataset. The Hilbert Transform can be used to transform real valued data into an analitical form, which we can use to explore the phase synchronisation. Take a look at [this](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.signal.hilbert.html) to see an implementation of the hilbert transform.

**Note:** It is good practice to always perform the hilbert transform in the demeaned data and to ignore the first and last time points (see this [paper]((http://online.liebertpub.com/doi/abs/10.1089/brain.2011.0068)) for an explantion). Luckly enough, due to the way our preprocessing is done if you load the filtered data the data is already demeaned.

In [3]:
# load the preprocessed extraced ROIs and calculate the Hilbert Transform
extracted_rois_path = '/home/jdafflon/code/msc_project/patrycja/data_out/extract_rois/rois_timeseries.txt'
extracted_rois = np.genfromtxt(extracted_rois_path, delimiter=',')

In [4]:
# Perform Hilbert tranform and exclude the first and last 10
# time points
hiltrans = hilbert(extracted_rois)
hiltrans = hiltrans[:, 10:-10]

Reduce the effects of noise on the dataset by using a small sliding window approach. To do this define a function that performs a sliding window on your hilbert transformed data

In [5]:
# Define the function using a sliding window of 5 time points
def sliding_window(hilbert_data, window_size):
    nregions = hilbert_data.shape[0]
    ntpoints = hilbert_data.shape[1]
    slided = np.zeros((nregions, ntpoints - window_size + 1), dtype=complex)
    window = np.ones(int(window_size)) / float(window_size)
    for roi in range(nregions):
        slided[roi, :] = np.convolve(hilbert_data[roi, :],
                                    window, 'valid')
    return slided

In [6]:
# Call your sliding window function
window_size = 5
slided_data = sliding_window(hiltrans, window_size)

## Synchrony and Metasbility

The phenomena of synchronisation have been well established to describe the physics behind chaotic systems and non-linear dynamics. Recently, however, this particular measure has been used to assess activation and dynamic transitions over different brain regions.

The phase synchrony describes how coherently the phase changes among different brain regions.

Synchrony is defined as:
    $$ R(t)e^{i\Phi(t)} = \frac{1}{N}\sum_{n=1}^{N} e^{i\Phi_n(t)}$$

Where N = the total number of regions being analysed. The level of synchrony between phase time-courses is described by R(t), in terms of how coherently phase changes over time. $\Phi_n(t)$ represents the phase angle of a specific region.

During fully synchronous behaviour, R(t) = 1 and 0 when phase across all phase time series is fully asynchronous.

To give you a feeling of what this mean, the animation below shows how the phase angle 3 toy regions (dots) change with time. The mean phase ($\frac{1}{N}\sum^{i\Phi_n}$) is represented by the arrow in the animation. As you can see, when the regions are synchronised the phase synchrony is higher when compared when they are not synchronised. 


<img src="images/animation.gif" width="400">

The formula above allows you to compute the mean phase at 2 levels:
    * Global: Where all regions are taken into account
    * Pairwise: The parameters of interest are calculated for each pair or regions

### Synchrony and Metastability

Use the formula above to calculate both pairwise as well as global synchrony.
Now, we want to extract distance from zero as this will tell us the strenght of the synchrony. We do this by taking the absolute value of hte complex signal.  


The metastability can be calculaated as the standard deviation of the synchrony. 

**Note**: remember that on the pairwise analyse you are only comparing two regions (N=2).

**Question**: How can you extract the phase from the hilbert transform? What will be the maximum and the minimum value of the phase?


In [7]:
# define matrix where pairwise synchrony will be saved
n_regions = slided_data.shape[0]
hilbert_t_points = slided_data.shape[1]


### Pairwise Synchrony

The next few lines will calculate the Synchrony values between pairs of regions

In [45]:
# Calculate the pairwise as well as the global synchrony for your extracted ROIs.
# get the phase angle for the data (all values should be between phi and -phi)
phase_angle = np.angle(slided_data)

# Find the indices of the pairs
rows, cols = np.indices((n_regions, n_regions))

# Where pair_synchrony metastability and mean_synchrony over all time points will
# be saved
pair_synchrony = np.zeros((n_regions, n_regions, hilbert_t_points), dtype=complex)
mean_synchrony = np.zeros((n_regions, n_regions))
pair_metastability = np.zeros((n_regions, n_regions))

for row in range(rows.shape[0]):
    for col in range(cols.shape[0]):
        pair_synchrony[row, col, :] = (np.exp(1j * (phase_angle[row, :])) + np.exp(1j * phase_angle[col, :]))/ 2
        mean_synchrony[row, col] = np.mean(abs(pair_synchrony[row, col, :]))
        pair_metastability[row, col] =  np.std(abs(pair_synchrony[row, col, :]))
        
pair_synchrony = abs(pair_synchrony)

**Note:** As you can see, the pair synchrony is a complex number. However, for this analysis we are only interested in the distance to zero as this corresponds to the magnitude of synchrony.

### Global Synchrony


Calculate Synchrony levels among all regions

In [61]:
phase_angle.shape

(82, 1176)

In [65]:
global_synchrony = 0
for region in range(n_regions):
    global_synchrony += abs(np.exp(phase_angle[region, :] * 1j))
global_synchrony /= n_regions

print global_synchrony

[ 1.  1.  1. ...,  1.  1.  1.]


In [60]:
global_synchrony = np.mean(np.tril(mean_synchrony), -1)
print global_synchrony
print global_synchrony.shape
global_metastability = np.std(global_synchrony)
print global_metastability

[ 0.01219512  0.02213908  0.03233474  0.04015713  0.04710536  0.05566733
  0.06040669  0.07451148  0.08646027  0.09760287  0.09658023  0.1130836
  0.12300425  0.11751356  0.13831043  0.15238926  0.15762255  0.16474401
  0.15871785  0.19200492  0.19046941  0.20740613  0.20970953  0.22732527
  0.23566734  0.2485468   0.25764943  0.26461126  0.27304445  0.28230531
  0.277085    0.30310514  0.30743525  0.31736643  0.3281598   0.33550628
  0.35546616  0.35592091  0.3618255   0.37489429  0.4081767   0.39341899
  0.40910977  0.3977794   0.43737933  0.39606546  0.44669044  0.44512781
  0.45991499  0.47400359  0.48872676  0.50924593  0.48879166  0.52552635
  0.53698535  0.54275325  0.54726021  0.57596168  0.58275584  0.58899072
  0.5902469   0.60958371  0.59754771  0.61296764  0.61158536  0.60903667
  0.6182809   0.63705243  0.66153059  0.6712462   0.68612902  0.69314376
  0.70873829  0.69402954  0.75239228  0.73366216  0.7448458   0.71798511
  0.72678808  0.77076361  0.78521143  0.77817978]
(8

### Save results

Save all results in a dictionary and then a pickle